In [1]:
!pip install pyDOE
!pip install cma

  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=0a35756ac0c865347da1a2262b07422a0f2eacd563c758df8f9599390ee5cf89
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 2.8MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import bbobbenchmarks as bn
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F9 '''
def F9(X):
    f = bn.F9()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data


''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))


''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 37.06977128135185 ,random_state=0 , l1_ratio=0.98989898989899, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 37.213462 , C = 1000.000000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_9_1000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv('test_9_200Samples.csv').iloc[:,1:]
true = np.array(test['Y'])

## Surrogate Models

In [0]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,51):
        Cols.append(Columns[j]+'_X'+str(i))
const = [ [-5] * 50, [5] * 50 ]

opt = cma.CMAOptions()
opt.set("bounds", const)
opt.set ("seed" , 0)
opt.set ("maxfevals" , 50000)

n_obs , dim =  30, 50
G = DOE(n_obs, dim)
G  = [ linearscaletransform(G[idx] , range_out=(-5,5)) for idx in range(dim) ]
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([30,150])
for i in range(G[1].shape[0]):
        min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
        min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
        min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
        X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimension 50 (seed=975031, Mon Apr 13 18:39:45 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     15 9.579423972707440e+05 1.0e+00 2.37e+00  2e+00  2e+00 0:00.0
    2     30 9.356504259902324e+05 1.0e+00 2.28e+00  2e+00  2e+00 0:00.0
    3     45 9.103635354186302e+05 1.0e+00 2.23e+00  2e+00  2e+00 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [1029951.90345591] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [955014.90872973] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1500 -6.383647569638507e+05 1.3e+00 2.78e-01  3e-01  3e-01 0:01.7
  200   3000 -6.965968863448370e+05 1.4e+00 1.63e-02  1e-02  2e-02 0:03.4
  300   4500 -6.968421515314265e+05 1.5e+00 9.56e-04  8e-04  9e-04 0:05.0
  400   6000 -6.968429685344961e+05 1.5e+00 4.89e-05  4e-05  4e-05 0:06.7
  500   7500 -6.968429697761713e+05 1.6e+00 2.59e-06  2e-06  2e-06 0:08.4
  600   9000 -6.968429697820235e+05 1.6e+00 1.48e-07  1e-07  1e-07 0:10.0
  700  10500 -6.968429697820356e+05 1.6e+00 1.69e-08  1e-08  1e-08 0:11.7
  800  12000 -6.968429697820356e+05 2.0e+00 1.02e-08  7e-09  9e-09 0:13.4
  900  13500 -6.968429697820351e+05 2.1e+00 7.70e-09  6e-09  6e-09 0:15.1
 1000  15000 -6.968429697820356e+05 2.5e+00 5.96e-09  4e-09  5e-09 0:16.8
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  16500 -6.968429697820356e+05 2.7e+00 4.48e-09  3e-09  4e-09 0:18.5
 1200  18000 -6.968429697820353e+05 2.8e+00 3.78e-09  3e-09  3e-09 0:20.1
 1300  19500 -6.968429697820353e+05 3.1e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [1011649.69745303] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=1011649.697453, sigma=2.36e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=1011649.697453, sigma=2.36e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=

  100   1500 7.000481246003249e+05 1.4e+00 3.02e-01  3e-01  3e-01 0:02.0
  200   3000 6.926925191847816e+05 1.6e+00 3.87e-02  3e-02  4e-02 0:04.0
  300   4500 6.926337474874050e+05 1.8e+00 2.94e-03  2e-03  3e-03 0:06.0
  400   6000 6.926333582532536e+05 1.8e+00 2.25e-04  2e-04  3e-04 0:08.0
  500   7500 6.926333559663679e+05 2.0e+00 1.64e-05  1e-05  2e-05 0:10.0
  600   9000 6.926333559525023e+05 2.1e+00 1.25e-06  9e-07  1e-06 0:11.9
  700  10500 6.926333559524220e+05 2.2e+00 1.21e-07  8e-08  1e-07 0:13.9
  800  12000 6.926333559524214e+05 2.3e+00 2.38e-08  2e-08  3e-08 0:15.9


...

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=692633.355952, sigma=2.27e-08).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=796)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=692633.355952, sigma=2.28e-08).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=797)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=692633.355952, sigma=2.36e-08).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=802)
  ')')
/usr/local/lib/pyth

  900  13500 6.926333559524214e+05 2.3e+00 1.83e-08  1e-08  2e-08 0:17.9


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=692633.355952, sigma=2.12e-08).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=940)
  ')')


  961  14415 6.926333559524214e+05 2.5e+00 1.65e-08  1e-08  2e-08 0:19.1
termination on tolfunhist=1e-12 (Mon Apr 13 19:22:33 2020)
final/bestever f-value = 6.926334e+05 6.926334e+05
incumbent solution: [ 0.21589879 -0.00611271  0.4668803   0.36328031 -1.06066457  1.14252787
 -0.05262523 -0.69366759 ...]
std deviations: [1.18514947e-08 1.07543001e-08 1.20067090e-08 1.26243204e-08
 1.38731341e-08 1.33176850e-08 1.36392544e-08 1.34046383e-08 ...]


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=692633.355952, sigma=1.68e-08).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=958)
  ')')


In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('Results\\F9_X_Values.csv')
Krig_Fun = np.zeros(30)
SVM_Fun = np.zeros(30)
ELN_Fun = np.zeros(30)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F9(X_Values.iloc[i,:50])
    SVM_Fun [i] = F9(X_Values.iloc[i,50:100])
    ELN_Fun [i] = F9(X_Values.iloc[i,100:150])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([1727.97816031]), count=array([1])) 4.344340065576707e-05
SVM
ModeResult(mode=array([614952.16518874]), count=array([1])) 417964.561769615
ELN
ModeResult(mode=array([1460.78564016]), count=array([1])) 0.00011237406206968753
